In [4]:
# imports
import pandas as pd
import plotly_express as px
import chart_studio.plotly as py
import chart_studio as cs
from urllib.parse import unquote
import plotly

def cleancountries(df):
   # cant find this region hence dropping
  df.loc[df['Country'] == 'Diamond Princess', ['Country']] = 'Cruise Ship'
  df['RecoveredCount'].fillna(0, inplace=True)

#   # these region names have to be changed which is understood by pycountry
#   df.loc[df['Region'] == 'Taiwan*', ['Region']] = 'Taiwan'
#   df.loc[df['Region'] == 'Korea, South', ['Region']] = 'Korea, Republic of'
#   df.loc[df['Region'] == 'Holy See', ['Region']] = 'Holy See (Vatican City State)'
#   df.loc[df['Region'] == 'Congo (Brazzaville)', ['Region']] = 'Congo'
#   df.loc[df['Region'] == 'Congo (Kinshasa)', ['Region']] = 'Congo, The Democratic Republic of the'
#   df.loc[df['Region'] == "Cote d'Ivoire", ['Region']] = 'Ivory Coast'
  return df

def getContinent(df):
    country_cont_map = pd.read_csv("mapping_countries.csv")
    df = pd.merge(df, country_cont_map, how='inner', on=['Country'])
    del df['Country']
    df.rename(columns={"Name": "Country"},inplace=True)
    print(df.columns)
    return df
    
confirmed_df = pd.read_csv(unquote(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv')))
del confirmed_df['Province/State']
confirmed_df.rename(columns={"Country/Region": "Country"},inplace=True)
confirmed_df = pd.melt(confirmed_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='ConfirmedCount')


recovered_df = pd.read_csv(unquote(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_recovered_global.csv&filename=time_series_covid19_recovered_global.csv')))
del recovered_df['Province/State']
recovered_df.rename(columns={"Country/Region": "Country"},inplace=True)
recovered_df = pd.melt(recovered_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='RecoveredCount')

death_df = pd.read_csv(unquote(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv')))
del death_df['Province/State']
death_df.rename(columns={"Country/Region": "Country"},inplace=True)
death_df = pd.melt(death_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='DeathCount')

finaldf = confirmed_df.merge(recovered_df,on=["Country",'Lat','Long','Day'],how='left').merge(death_df,on=["Country",'Lat','Long','Day'],how='left')
del finaldf['Lat']
del finaldf['Long']

finaldf['Day'] = pd.to_datetime(finaldf['Day'])
finaldf = cleancountries(finaldf)
finaldf = getContinent(finaldf)

group = finaldf.groupby(['Continent','Country','Day'])['RecoveredCount','DeathCount','ConfirmedCount'].sum().reset_index().sort_values('Day')
group['Day'] = group['Day'].astype(str)
fig = px.scatter(group,
           x="RecoveredCount",
           y="DeathCount",
           size="ConfirmedCount",
           size_max=60,
           color = 'Continent',
           hover_name="Country",
           animation_frame="Day",
           animation_group="Country",
           log_x=True,
           range_x=[100, 100000],
           range_y=[0, 35000])
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False,title="Covid-19 Spread Globally",xaxis_title="Recovered Cases",yaxis_title="Death Cases")
plotly.offline.plot(fig, filename='Covid-19 Spread Acceleration.html')
# fig.show()

Index(['Day', 'ConfirmedCount', 'RecoveredCount', 'DeathCount', 'Country',
       'Continent'],
      dtype='object')


c:\personal\bubble-animated\viz-env\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



'Covid-19 Spread Acceleration.html'